In [8]:
path = "dataset"

In [9]:
!pip install stemming
!pip install nltk
# !pip install StringIO

In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
import numpy as np
import nltk
import re
from sklearn.cluster import KMeans
import csv

In [11]:
train_data = pd.read_csv('{}/news.csv'.format(path))

In [12]:
train_data.head()

,id,headline,text
0,uid-1,Market Advances 5.12 More,NEW YORK (AP) - A prime rate reduction by New ...
1,uid-2,District Boosts Request For Anti-Terrorism Aid...,Mayor Anthony A. Williams petitioned the White...
2,uid-3,"Election? Here's How You Do It, Mate.",From our downunder perspective here in Austral...
3,uid-4,The Biggest Boom Ever,We are about to rewrite history. Unless a rece...
4,uid-5,Economic Aide Sees Uptrend,"Sedate and scholarly Dr. Arthur Burns, the ex-..."


In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/vkv/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
data = train_data.fillna('NULL')

In [15]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [16]:
vectorizer = TfidfVectorizer(max_df=0.80, max_features=200000,
                                 min_df=0.07, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))


counts = vectorizer.fit_transform(data['headline'])
kmeans = KMeans(n_clusters=9, init='k-means++', max_iter=100, n_init=1)
    
transformed = kmeans.fit_transform(counts)
data['cluster_num'] = kmeans.fit_predict(counts)

label = kmeans.labels_
sil_coeff = silhouette_score(counts, label, metric='euclidean')

/Users/vkv/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  sorted(inconsistent))


In [17]:
print(transformed)
np.savetxt('prediction2.txt', np.array(transformed))

[[0.         1.         1.         ... 1.         1.         1.        ]
 [0.         1.         1.         ... 1.         1.         1.        ]
 [1.         0.         1.41421356 ... 1.41421356 0.95224851 0.50171421]
 ...
 [1.         0.         1.41421356 ... 1.41421356 0.95224851 0.50171421]
 [0.         1.         1.         ... 1.         1.         1.        ]
 [0.         1.         1.         ... 1.         1.         1.        ]]


In [18]:

with open('prediction2.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["id", "cluster"])
    for index, row in data.iterrows():
        Y = vectorizer.transform([row['headline']])
        prediction = kmeans.predict(Y)
        writer.writerow([row['id'], prediction[0]])